New type test - a custom type with its own validators

In [1]:
from collections import namedtuple

from schematics.exceptions import DataError, ValidationError, ConversionError
from schematics.models import Model
from schematics.types import *

In [2]:
Dimensions = namedtuple("Dimensions", "length, width, height")

In [3]:
class DimensionsType(BaseType):
    def __init__(self, *args, max_largest_dimension=None, max_volume=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.max_largest_dimension = max_largest_dimension
        self.max_volume = max_volume
    
    def to_native(self, value, context=None):
        try:
            return Dimensions._make(value)
        except (TypeError, ValueError) as e:
            raise ConversionError(e.args)
            
    def to_primitive(self, value, context=None):
        return tuple(value)
    
    def validate_max_largest_dimension(self, value):
        constraint = self.max_largest_dimension
        if constraint is None:
            return
        if max(value) > constraint:
            raise ValidationError(f'Largest dimension is greater than maximum of {constraint}')
            
    def validate_max_volume(self, value):
        constraint = self.max_volume
        if constraint is None:
            return
        volume = value.length * value.width * value.height
        if volume > constraint:
            raise ValidationError(f'Volume is greater than maximum of {constraint}')

In [4]:
class Box(Model):
    size = DimensionsType(
        required=True,
        max_largest_dimension=10,
        max_volume=250
    )

In [5]:
box = Box({"size": (8, 5, 6)})
box.validate()
box.size

Dimensions(length=8, width=5, height=6)

In [6]:
box = Box({"size": (11, 0.5, 0.3)})
try:
    box.validate()
except DataError as e:
    print(e.errors)

{'size': ValidationError([ErrorMessage('Largest dimension is greater than maximum of 10', None)])}


In [7]:
box = Box({"size": (8, 5, 7)})
try:
    box.validate()
except DataError as e:
    print(e.errors)

{'size': ValidationError([ErrorMessage('Volume is greater than maximum of 250', None)])}


In [8]:
try:
    box = Box({"size": (1, 2)})
except DataError as e:
    print(e.errors)

{'size': ConversionError([ErrorMessage('Expected 3 arguments, got 2', None)])}
